# Wikipedia Notable Life Expectancies

# [Notebook 2 of 4: Data Cleaning](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wp_life_expect_data_clean_thanak_2022_06_13.ipynb)

## Context

The


## Objective

The

### Data Dictionary

- Feature: Description

## Importing Necessary Libraries

In [1]:
# To structure code automatically
%load_ext nb_black

# To import/export sqlite databases
import sqlite3 as sql

# To help with reading and manipulating data
import pandas as pd
import numpy as np
import re

# To define maximum number of columns to be displayed in a dataframe
pd.set_option("display.max_columns", None)
# To define the maximum number of rows to be displayed in a dataframe
pd.set_option("display.max_rows", 200)

# To supress warnings
# import warnings

# warnings.filterwarnings("ignore")

# To set some visualization attributes
pd.set_option("max_colwidth", 150)

<IPython.core.display.Javascript object>

## Data Overview

### Reading, Sampling, and Checking Data Shape

In [2]:
# Reading the wp_life_expect_raw_complete dataset
conn = sql.connect("wp_life_expect_raw_complete.db")
data = pd.read_sql("SELECT * FROM wp_life_expect_raw_complete", conn)

# Making a working copy
df = data.copy()

# Checking the shape
print(f"There are {df.shape[0]} rows and {df.shape[1]} columns.")

# Checking first 2 rows of the data
df.head(2)

There are 133900 rows and 6 columns.


,month_year,day,name,info,link,num_references
0,January 1994,1,William Chappell,", 86, British dancer, ballet designer and director.",https://en.wikipedia.org/wiki/William_Chappell_(dancer),21
1,January 1994,1,Raymond Crotty,", 68, Irish economist, writer, and academic.",https://en.wikipedia.org/wiki/Raymond_Crotty,12


<IPython.core.display.Javascript object>

In [3]:
# Checking last 2 rows of the data
df.tail(2)

,month_year,day,name,info,link,num_references
133898,June 2022,9,Oleg Moliboga,", 69, Russian volleyball player, Olympic champion (1980) and coach.",https://en.wikipedia.org/wiki/Oleg_Moliboga,2
133899,June 2022,9,Zou Jing,", 86, Chinese engineer, member of the Chinese Academy of Engineering.",https://en.wikipedia.org/wiki/Zou_Jing_(engineer),3


<IPython.core.display.Javascript object>

In [4]:
# Checking a sample of the data
df.sample(5)

,month_year,day,name,info,link,num_references
9731,October 1997,3,Blake Wayne Van Leer,", 71, United States Navy officer.",https://en.wikipedia.org/wiki/Blake_Wayne_Van_Leer,20
94131,January 2018,4,Raymond Robinson,", 88, South African Olympian [11]",https://en.wikipedia.org/wiki/Raymond_Robinson_(cyclist),2
59648,January 2013,1,Alois Moser,", 82, Canadian Olympic ski jumper (1960).",https://en.wikipedia.org/wiki/Alois_Moser,2
88294,February 2017,24,Lee Tsuntung,", 100, Chinese Olympic basketball player (1948).",https://en.wikipedia.org/wiki/Lee_Tsuntung,2
91517,August 2017,15,Joe McGurn,", 52, Scottish footballer (St Johnstone, Alloa Athletic, Stenhousemuir), cancer.",https://en.wikipedia.org/wiki/Joe_McGurn,3


<IPython.core.display.Javascript object>

#### Observations:
- There are 133,900 rows and 6 columns.
- `month_year` contains the month and year of death, while `day` contains the day of the month of death.
- `name` is the notable person's name.  It is a nominal feature that will not be used for analysis, but will be maintained for any referencing needs.
- `info` contains multiple items including the notable person's "age, country of citizenship at birth, subsequent country of citizenship (if applicable), reason for notability, (and) cause of death (if known)."
- `link` is the url to the notable person's individual Wikipedia page.  If such a page does not exist, there is either a non-working link (https://en.wikipedia.orgNone), or the link is to a page with a message that the page does not exist for that individual.  `link` is a unique identifier for all entries, except the 6 with the non-working link, which do have unique `name` values from each other.
- `num_references` contains the number of references on the notable person's individual Wikipedia page.  This feature serves as a proxy measure of notability.
- Prior to EDA, our task will be to extract the individual elements that are comined in `month_year` and `info` columns.

### Checking Data Types, Duplicates, and Null Values

In [5]:
# Checking data types and null values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133900 entries, 0 to 133899
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   month_year      133900 non-null  object
 1   day             133900 non-null  object
 2   name            133894 non-null  object
 3   info            133900 non-null  object
 4   link            133900 non-null  object
 5   num_references  133900 non-null  object
dtypes: object(6)
memory usage: 6.1+ MB


<IPython.core.display.Javascript object>

In [6]:
# Checking duplicate rows
df.duplicated().sum()

0

<IPython.core.display.Javascript object>

In [7]:
# Check percentage of null values by column
df.isnull().sum() / df.count() * 100

month_year        0.000000
day               0.000000
name              0.004481
info              0.000000
link              0.000000
num_references    0.000000
dtype: float64

<IPython.core.display.Javascript object>

In [8]:
# Checking number of missing values per row (not necessary here, but done to keep process standard)
df.isnull().sum(axis=1).value_counts()

0    133894
1         6
dtype: int64

<IPython.core.display.Javascript object>

#### Observations:
- Our dataset was saved to and read from the database without any hiccups.
- As expected, we have 6 entries that are missing `name`, but we will find it in their `info` values.
- All columns are currently of object type.  We will need to appropriately typecast them after separating the information in `month_year` and `info`.

## Data Cleaning

### Addressing Missing `name` Values

In [9]:
# Checking rows with missing name values
missing_name = df[df["name"].isna()]
missing_name

,month_year,day,name,info,link,num_references
18937,August 2001,11,None,"Kevin Kowalcyk, 2, known for eating a hamburger contaminated with E. coli O157:H7.",https://en.wikipedia.orgNone,0
24985,January 2004,22,None,"Vincent Palmer, 37, British criminal.",https://en.wikipedia.orgNone,0
27458,March 2005,1,None,"Barry Stigler, 57, American voice actor.",https://en.wikipedia.orgNone,0
34077,July 2007,11,None,"Nana Gualdi, 75, German singer and actress.",https://en.wikipedia.orgNone,0
64769,September 2013,29,None,"Scott Workman, 47, American stuntman (, , ), cancer.",https://en.wikipedia.orgNone,0
106613,September 2019,12,None,"Thami Shobede, 31, Singer Songwriter",https://en.wikipedia.orgNone,0


<IPython.core.display.Javascript object>

#### Observations:
- These rows vary from the main set as there is a substring containing the person's name at the start of the `info` string.
- As there are so few rows missing `name`, let us address this issue first.

In [10]:
# For loop to copy name value from info value and remove name from info value
treat_rows = missing_name.index
for i in treat_rows:
    info = df.loc[i, "info"]
    info_lst = info.split(sep=",", maxsplit=1)

    name = info_lst[0].strip()
    df.loc[i, "name"] = name
    df.loc[i, "info"] = re.sub(name, "", info).strip()

# Re-check rows
df.loc[treat_rows, :]

,month_year,day,name,info,link,num_references
18937,August 2001,11,Kevin Kowalcyk,", 2, known for eating a hamburger contaminated with E. coli O157:H7.",https://en.wikipedia.orgNone,0
24985,January 2004,22,Vincent Palmer,", 37, British criminal.",https://en.wikipedia.orgNone,0
27458,March 2005,1,Barry Stigler,", 57, American voice actor.",https://en.wikipedia.orgNone,0
34077,July 2007,11,Nana Gualdi,", 75, German singer and actress.",https://en.wikipedia.orgNone,0
64769,September 2013,29,Scott Workman,", 47, American stuntman (, , ), cancer.",https://en.wikipedia.orgNone,0
106613,September 2019,12,Thami Shobede,", 31, Singer Songwriter",https://en.wikipedia.orgNone,0


<IPython.core.display.Javascript object>

#### Observations:
- Missing `name` values have been addressed and those names have been removed from `info` values.

In [11]:
# Re-check info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133900 entries, 0 to 133899
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   month_year      133900 non-null  object
 1   day             133900 non-null  object
 2   name            133900 non-null  object
 3   info            133900 non-null  object
 4   link            133900 non-null  object
 5   num_references  133900 non-null  object
dtypes: object(6)
memory usage: 6.1+ MB


<IPython.core.display.Javascript object>

#### Observations:
- We have no remaining missing values.
- Let us treat the `month_year` column next.

### Separating `month` and `year`

In [12]:
# Separating month and year into 2 columns and typecasting year as integer
df.loc[:, "year"] = df["month_year"].apply(lambda x: x.split(sep=" ")[1].strip())
df["year"] = df["year"].apply(lambda x: int(x))

df.loc[:, "month"] = df["month_year"].apply(lambda x: x.split(sep=" ")[0])
df.head(2)

,month_year,day,name,info,link,num_references,year,month
0,January 1994,1,William Chappell,", 86, British dancer, ballet designer and director.",https://en.wikipedia.org/wiki/William_Chappell_(dancer),21,1994,January
1,January 1994,1,Raymond Crotty,", 68, Irish economist, writer, and academic.",https://en.wikipedia.org/wiki/Raymond_Crotty,12,1994,January


<IPython.core.display.Javascript object>

In [13]:
# Dropping month_year column
df.drop("month_year", axis=1, inplace=True)
df.head(2)

,day,name,info,link,num_references,year,month
0,1,William Chappell,", 86, British dancer, ballet designer and director.",https://en.wikipedia.org/wiki/William_Chappell_(dancer),21,1994,January
1,1,Raymond Crotty,", 68, Irish economist, writer, and academic.",https://en.wikipedia.org/wiki/Raymond_Crotty,12,1994,January


<IPython.core.display.Javascript object>

### Treating `info`

#### Checking a Sample

In [14]:
# Checking a sample of info
df.sample(5)

,day,name,info,link,num_references,year,month
108995,6,Oswaldo Larriva,", 74, Ecuadorian academic and politician, Governor of Azuay Province (2007–2009, 2019) and Deputy (1992–1994, 2013–2017), leukaemia.",https://en.wikipedia.org/wiki/Oswaldo_Larriva,1,2020,January
70950,14,George V. Hansen,", 83, American politician, member of the U.S. House for Idaho's 2nd district (1965–1969, 1975–1985).",https://en.wikipedia.org/wiki/George_V._Hansen,9,2014,August
2615,3,Andrei Nikolsky,", 36, Russian pianist, traffic collision.",https://en.wikipedia.org/wiki/Andrei_Nikolsky,0,1995,February
54584,14,Ernst-Günter Habig,", 76, German Olympic footballer, stroke.",https://en.wikipedia.org/wiki/Ernst-G%C3%BCnter_Habig,5,2012,March
28663,13,Donald Howard Shively,", 84, American professor, among the first to promote modern East Asian Studies, Shy–Drager syndrome.",https://en.wikipedia.org/wiki/Donald_Shively,4,2005,August


<IPython.core.display.Javascript object>

#### Observations:
- First, let us check for any rows that are missing digits, and therefore the age target, within `info` and remove them.
- Also, it would be helpful to remove information contained within parentheses, as we will not be using it.

#### Checking and Dropping Rows Lacking Digits (and therefore Age Data) within `info`

In [15]:
# For loop to extract index of rows without digits in info value
remove_lst = []
for index in df.index:
    pattern = r"\d"
    if re.search(pattern, df.loc[index, "info"]) is None:
        remove_lst.append(index)
print(len(remove_lst), "rows")
df.loc[remove_lst, :].sample(5)

925 rows


,day,name,info,link,num_references,year,month
13297,2,Kamel Sajid,", Iraqi general of the Republican Guard, executed.",https://en.wikipedia.org/wiki/Kamel_Sajid,3,1999,March
90058,26,Sabzar Bhat,", Indian militant commander (Hizbul Mujahideen), shot.",https://en.wikipedia.org/wiki/Sabzar_Bhat,22,2017,May
28962,24,Bala Usman,", Nigerian academic, politician and historian.",https://en.wikipedia.org/wiki/Bala_Usman,7,2005,September
32834,8,Ismail Semed,", Chinese Muslim Uighur separatist, execution by firing squad.",https://en.wikipedia.org/wiki/Ismail_Semed,6,2007,February
70383,15,Abdukadir Osman,", Somali writer.",https://en.wikipedia.org/wiki/Abdukadir_Osman,1,2014,July


<IPython.core.display.Javascript object>

In [16]:
# Dropping rows missing age data
df.drop(remove_lst, inplace=True)
df.reset_index(inplace=True, drop=True)
df.shape

(132975, 7)

<IPython.core.display.Javascript object>

#### Observations:
- 925 rows were removed as they lacked the target data for `age`.

#### Removing Information within Parentheses in `info`

In [17]:
# Regular expression for parenthesis and its contents
pattern = r"\(.*\)"

# Subbing empty string for parentheses and stripping white space
df.loc[:, "info"] = df["info"].apply(lambda x: re.sub(pattern, "", x).strip())
df.loc[[21850, 125430], :]

,day,name,info,link,num_references,year,month
21850,11,Sir Michael Clapham,", 90, British industrialist, president of the Confederation of British Industry from 1972 to 1974.",https://en.wikipedia.org/wiki/Michael_Clapham_(industrialist),14,2002,November
125430,23,Jean-Luc Nancy,", 81, French philosopher.",https://en.wikipedia.org/wiki/Jean-Luc_Nancy,8,2021,August


<IPython.core.display.Javascript object>

#### Observation:
- Parentheses and information within has been removed from `info`.
- Next, we will follow the Wikipedia-defined fields to divide the `info` values.

#### Splitting `info` on Commas into Separate Columns

In [18]:
# For loop to split info on commas and separate into respective new columns and removing leading/trailing white space and periods
for i, item in enumerate(df["info"]):
    info_lst = item.split(",")

    for j in range(len(info_lst)):
        df.loc[i, f"info_{j}"] = info_lst[j].strip(" .")

# Checking the first 2 rows
df.head(2)

,day,name,info,link,num_references,year,month,info_0,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11
0,1,William Chappell,", 86, British dancer, ballet designer and director.",https://en.wikipedia.org/wiki/William_Chappell_(dancer),21,1994,January,,86,British dancer,ballet designer and director,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Raymond Crotty,", 68, Irish economist, writer, and academic.",https://en.wikipedia.org/wiki/Raymond_Crotty,12,1994,January,,68,Irish economist,writer,and academic,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<IPython.core.display.Javascript object>

In [19]:
# Checking the last 2 rows
df.tail(2)

,day,name,info,link,num_references,year,month,info_0,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11
132973,9,Oleg Moliboga,", 69, Russian volleyball player, Olympic champion and coach.",https://en.wikipedia.org/wiki/Oleg_Moliboga,2,2022,June,,69,Russian volleyball player,Olympic champion and coach,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
132974,9,Zou Jing,", 86, Chinese engineer, member of the Chinese Academy of Engineering.",https://en.wikipedia.org/wiki/Zou_Jing_(engineer),3,2022,June,,86,Chinese engineer,member of the Chinese Academy of Engineering,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<IPython.core.display.Javascript object>

#### Observations:
- The `info` value is successfully divided and we can proceed through it column by column.
- We will check the set of values for the first two columns, for age.

#### `info_0`

In [20]:
# Checking unique value counts
df["info_0"].value_counts()

                                                                       132946
Sir                                                                         6
92                                                                          2
Douglas Scott                                                               1
VC                                                                          1
Sir Woolwich West                                                           1
Sir Lord Justice of Appeal                                                  1
Dame MEP                                                                    1
83                                                                          1
Sir Governor-General                                                        1
Notable ice hockey players and coaches among the 44 killed in the :         1
Mike Alexander                                                              1
Colonel                                                         

<IPython.core.display.Javascript object>

#### Observations:
- The vast majority of rows have an empty string for this field.
- There is one row representing a group, rather than an individual, and we will drop it.
- We should verify the name and age information for the remainder of unique values in `info_0`.

#### Dropping Entry for Group

In [21]:
# Checking the entry representing a group
group_entry = df[
    df["info_0"]
    == "Notable ice hockey players and coaches among the 44 killed in the :"
]
group_entry

,day,name,info,link,num_references,year,month,info_0,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11
51130,7,2011 Lokomotiv Yaroslavl plane crash,Notable ice hockey players and coaches among the 44 killed in the :,https://en.wikipedia.org/wiki/2011_Lokomotiv_Yaroslavl_plane_crash,95,2011,September,Notable ice hockey players and coaches among the 44 killed in the :,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<IPython.core.display.Javascript object>

In [22]:
# Dropping group entry
df.drop(group_entry.index, inplace=True)
df.reset_index(inplace=True, drop=True)

<IPython.core.display.Javascript object>

In [24]:
list_to_check = df["info_0"].value_counts().index.to_list()

verify_df = pd.DataFrame()
for item in list_to_check[1:]:
    verify_df = pd.concat([verify_df, df[df["info_0"] == item]])
verify_df

,day,name,info,link,num_references,year,month,info_0,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11
30349,25,Julian Bullard,"Sir , 78, British diplomat.",https://en.wikipedia.org/wiki/Julian_Bullard,3,2006,May,Sir,78,British diplomat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31140,1,Kyffin Williams,"Sir , 88, Welsh artist, lung and prostate cancer.",https://en.wikipedia.org/wiki/Kyffin_Williams,15,2006,September,Sir,88,Welsh artist,lung and prostate cancer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34207,15,Jeremy Moore,"Sir , 79, British soldier, commander of UK land forces in the Falklands War.",https://en.wikipedia.org/wiki/Jeremy_Moore,3,2007,September,Sir,79,British soldier,commander of UK land forces in the Falklands War,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43606,29,Derek Hodgkinson,"Sir , 92, British air chief marshal.",https://en.wikipedia.org/wiki/Derek_Hodgkinson,5,2010,January,Sir,92,British air chief marshal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67212,7,Richard Best,"Sir , 80, British diplomat, Ambassador to Iceland .",https://en.wikipedia.org/wiki/Richard_Best_(diplomat),5,2014,March,Sir,80,British diplomat,Ambassador to Iceland,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67217,7,Thomas Hinde,"Sir , 88, British novelist.",https://en.wikipedia.org/wiki/Thomas_Hinde_(novelist),13,2014,March,Sir,88,British novelist,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33086,28,David Turnbull,". 92, American materials scientist.",https://en.wikipedia.org/wiki/David_Turnbull_(materials_scientist),8,2007,April,92,American materials scientist,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
117387,11,Gotthilf Fischer,"92, German choral conductor .",https://en.wikipedia.org/wiki/Gotthilf_Fischer,4,2020,December,92,German choral conductor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67517,21,Colin Turner,"Sir Woolwich West, 92, British politician, MP for .",https://en.wikipedia.org/wiki/Colin_Turner,3,2014,March,Sir Woolwich West,92,British politician,MP for,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67162,5,Robin Dunn,"Sir Lord Justice of Appeal, 96, British jurist, .",https://en.wikipedia.org/wiki/Robin_Dunn,3,2014,March,Sir Lord Justice of Appeal,96,British jurist,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<IPython.core.display.Javascript object>

#### Observations:
- We can extract the ages, from rows in which they occur in `info_0` instead of `info_1`.
- The majority of rows contain additional aliases or titles within `info_0`, that we don't need.  
- There are few rows that will need to be treated individually to correct the name value, as follows:
    1. 42140 entry is for Mike Alexander whose band was Evile.
    2. 14035 entry is for Herbert Wiere who performed slapstick.
    3. 30210 entry is for Sarah-Jayne Mulvihill who was a Flight Lieutenant.
    4. 18312 entry is for Douglass Scott who was killed by Demetreus Nix.
    5. 15873 entry is for Kim Hwan-Sung who was a member of the band NRG.
  We can replace the `name` value with the `info_0` value for these rows
- Row 18908 represents a group, rather than an individual, so will be dropped, after treating the above rows.

Extract ages including Nearly 3, Herbert Wiere, Kim Hwan-Sung, Douglass Scott (not Demetreus Nix), Sarah-Jayne Mulvihill (flight lieutenant), Mike Alexander (not Evile)

In [20]:
# Dropping info column
df.drop("info", axis=1, inplace=True)

<IPython.core.display.Javascript object>

#### Checking the Set of First Substrings (Before First Comma)

In [ ]:
df["info_1"].unique()

In [ ]:
# Checking values in the first substring of info (before the first comma)
substring_set = set(df["info"].apply(lambda x: x.split(sep=",", maxsplit=1)[0]))
substring_set

#### Observations:
- Though we can see the age values, there is a lot of other information in the first substring of the `info` value--1412 variations.
- Some entries may be missing the age information.
- Also, age is entered in various formats.
- Let us examine the rows that start with a digit.

In [ ]:
# Checking the different patterns of first substrings that start with digits
{item for item in substring_set if re.match(r"\d", item)}

#### Observations:
- The patterns include "age", "age ", "age+", "age/", "age?, "age–", "age months", "age-" (distinct dash),"age-months", "age–months" (distinct dash), "ages", "age?", "age days", "age."
- We will need to address ages in months first.

#### Extracting `age` from `info` for `info` Values that Begin with "age,"

In [ ]:
# Creating a list of ages
ages = pd.Series(np.arange(1, 150)).astype("string")

# For loop to extract age from info column and assign to age column, if info starts with 'age,'
for age in ages:
    for i, item in enumerate(df["info"]):
        if item.startswith(age + ","):
            df.loc[i, "age"] = int(age)
df.sample(5)

In [ ]:
# Checking for remaining rows missing age
df["age"].isna().sum()

#### Observations:
- We were able to extract `age` for the vast majority of entries.
- Let us take a look at the remaining entries.

#### Checking Rows with Missing `age`

In [ ]:
df[df["age"].isna()]

#### Observations:
- We can immediately see 2 apparent patterns.  The first is an age range with a hyphen and the second is age missing altogether.
- Let us do another iteration for the age-range pattern, accepting the lower value as age.

#### Extracting `age` from `info` for `info` Values that Begin with "age-"

In [ ]:
# Creating a list of ages
ages = pd.Series(np.arange(1, 150)).astype("string")

# For loop to extract age from info column and assign to age column, if info starts with 'age-'
for age in ages:
    for i, item in enumerate(df["info"]):
        if item.startswith(age + "-"):
            df.loc[i, "age"] = int(age)
df.sample(5)

In [ ]:
# Checking the number of remaining missing values for `age`
df["age"].isna().sum()

#### Observation:
- The last iteration captured nearly 200 values.
- Let us take a look at the remaining rows with missing `name'.

#### Checking Rows with Missing `age`

In [ ]:
# Checking rows still missing `age`
df[df["age"].isna()].sample(5)

#### Observations:
- It almost appears as if our iteration for addressing `info` starting with "age-' missed some values.
- Closer examination reveals that the dash character varies for the remaining values ("-" vs "–").
- So we will iterate again with the larger dash.

In [ ]:
# Creating a list of ages
ages = pd.Series(np.arange(1, 150)).astype("string")

# For loop to extract age from info column and assign to age column, if info starts with 'age-'
for age in ages:
    for i, item in enumerate(df["info"]):
        if item.startswith(age + "–"):
            df.loc[i, "age"] = int(age)
df.sample(5)

In [ ]:
# Checking the number of remaining missing values for `age`
df["age"].isna().sum()

#### Observation:
- That iteration captured over 400 values.
- Let us take a look at the remaining rows with missing `name'.

#### Checking Rows with Missing `age`

In [ ]:
# Checking rows still missing `age`
df[df["age"].isna()].head(100)

#### Observations:
- The next pattern starts with 'age/', so we will iterate again with it.

In [ ]:
# Creating a list of ages
ages = pd.Series(np.arange(1, 150)).astype("string")

# For loop to extract age from info column and assign to age column, if info starts with 'age-'
for age in ages:
    for i, item in enumerate(df["info"]):
        if item.startswith(age + "/"):
            df.loc[i, "age"] = int(age)
df.sample(5)

In [ ]:
# Checking the number of remaining missing values for `age`
df["age"].isna().sum()

#### Observation:
- That iteration captured nearly 70 missing values.
- Let us take a look at the remaining rows with missing `name'.

#### Checking Rows with Missing `age`

In [ ]:
# Checking rows still missing `age`
df[df["age"].isna()].head(100)

#### Observations:
- The next pattern starts with 'age.', so we will iterate again with it.

In [ ]:
# Creating a list of ages
ages = pd.Series(np.arange(1, 150)).astype("string")

# For loop to extract age from info column and assign to age column, if info starts with 'age-'
for age in ages:
    for i, item in enumerate(df["info"]):
        if item.startswith(age + "."):
            df.loc[i, "age"] = int(age)
df.sample(5)

In [ ]:
# Checking the number of remaining missing values for `age`
df["age"].isna().sum()

#### Observation:
- We captured a handful of more missing values with that iteration.
- Let us take a look at the remaining rows with missing `name'.

#### Checking Rows with Missing `age`

In [ ]:
# Checking rows still missing `age`
df[df["age"].isna()].head(100)

#### Observations:
- There is at least one entry that escaped the net of that last iteration.  It can be hard-coded later.
- The next pattern starts with "age ", so will will iterate again with it.

In [ ]:
# Creating a list of ages
ages = pd.Series(np.arange(1, 150)).astype("string")

# For loop to extract age from info column and assign to age column, if info starts with 'age-'
for age in ages:
    for i, item in enumerate(df["info"]):
        if item.startswith(age + " "):
            df.loc[i, "age"] = int(age)
df.sample(5)

In [ ]:
# Checking the number of remaining missing values for `age`
df["age"].isna().sum()

#### Observations:
- That iteration captured 26 more missing values.
- Let us look again at the remaining rows.

#### Checking Rows with Missing `age`

In [ ]:
# Checking rows still missing `age`
df[df["age"].isna()].head(100)

#### Observations:
- This sample shows entries that are missing age information, or the age is imbedded in the middle of the string. 
- Let us remove rows without any age information, as they will not add to the analysis.

In [ ]:
# Creating a list of ages
ages = pd.Series(np.arange(1, 150)).astype("string")

# List of index of entries to drop due to no age information
drop_list = []

# For loop to add index of entries to drop_list if no age information present
for i in df[df["age"].isna()].index:
    if not any(age in df.loc[i, "info"] for age in ages):
        drop_list.append(i)
df.loc[drop_list, :].sample(50)

In [ ]:
# Checking the number of entries that will be dropped
print(len(drop_list))

# Dropping the rows without age information
df.drop(drop_list, axis=0, inplace=True)

In [ ]:
# Checking df shape after dropping rows
df.shape

#### Observations:
- We have successfully dropped the entries lacking age data.
- Let us examine the remaining rows missing `age`.

#### Checking Rows with Missing `age`

In [ ]:
# Checking rows still missing `age`
df[df["age"].isna()].sample(100)

#### Removing Leading and Trailing Commas, Whitespace, and Periods

In [ ]:
# Removing the leading/trailing commas, periods, and whitespace
df.loc[:, "info"] = df["info"].apply(lambda x: x.strip(" ,."))
df.sample(5)